In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import pandas as pd
import numpy as np
from quantfreedom.class_practice.enums import *
from quantfreedom.class_practice.base import backtest_df_only, plot_one_result


price_data = pd.read_hdf("../../tests/data/4hBTCETH.hd5")
%load_ext autoreload
%autoreload 2


In [3]:
entries_array = np.array(
    [
        [False, False, True, True, False, False],
        [True, True, True, True, False, False],
    ]
).T
entries = pd.DataFrame(entries_array, index=price_data.index)
exit_signals = np.array(
    [
        [False, False, False, False, False, True],
        [False, False, False, False, True, False],
    ]
).T
exit_signals = pd.DataFrame(exit_signals, index=price_data.index)


In [4]:
entries_array

array([[False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [False, False],
       [False, False]])

In [5]:
account_state = AccountState()
backtest_settings = BacktestSettings()
exchange_settings = ExchangeSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([30]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array(
        [IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([100]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.High]),
    sl_to_be_when_pct_from_candle_body=np.array([1, 2]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.ZeroLoss]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.Close]),
    trail_sl_when_pct_from_candle_body=np.array([2.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([1.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
)

In [6]:
order_records_df, strat_res_df, order_set_res_df = backtest_df_only(
    account_state=account_state,
    order_settings_arrays=order_settings_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data,
    entries=entries,
)

Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 2
Total indicator settings per symbol: 1
Total indicator settings to test: 2
Total order settings per symbol: 4
Total order settings to test: 8
Total candles per symbol: 6
Total candles to test: 48

Total combinations to test: 8


In [11]:
plot_one_result(
    strat_result_index=0,
    strat_res_df=strat_res_df,
    price_data=price_data,
    order_records_df=order_records_df,
)